<a href="https://colab.research.google.com/github/RedPlunder/CS450-Winter/blob/main/RAG_and_Prompt_test_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**RAG version: 1.0**

**Prompt version: 3.0**

**gpt version: 4o-mini**

**key improvements: new XML prompt**

In [ ]:
!pip install faiss-cpu

In [ ]:
import pandas as pd
import openai
from transformers import AutoTokenizer, AutoModel
import faiss
import numpy as np
import os
import torch
import asyncio
from google.colab import userdata

### Setup

In [ ]:
llm_choice = "gpt"
openai.api_key = userdata.get("OPENAI_API_KEY")
client = openai.AsyncClient(api_key=userdata.get("OPENAI_API_KEY"))

# Set environment variable to prevent runtime issues
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Load tokenizer and model for embedding
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L12-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L12-v2")

# gpu for embed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")

Using device: cuda




```
# This is formatted as code
```

### Load knowledge data

In [ ]:
# Load knowledge database
rag_db = pd.read_csv("./dataset/kd.csv")
rag_data = rag_db[['ID', 'Content']].dropna()

# Load knowledge database
rag_data['Content'] = rag_data['Content'].str.lower()
documents = rag_data['Content'].tolist()
doc_ids = rag_data['ID'].tolist()

### embed knowledge text

In [ ]:


def embed(documents, batch_size=20):
    """Generate embeddings in batches to prevent memory overflow."""
    all_embeddings = []
    for i in range(0, len(documents), batch_size):
        print(f"embedding {i} / {len(documents)}")
        batch = documents[i:i+batch_size]
        # Tokenize the batch with dynamic padding
        inputs = tokenizer(batch, padding="longest", truncation=True, max_length=512, return_tensors="pt")
        # move input to GPU
        inputs = {key: value.to(device) for key, value in inputs.items()}
        # Disable gradient calculation for inference
        with torch.no_grad():
            outputs = model(**inputs)
        #
        embeddings = outputs.last_hidden_state[:, 0, :].detach().cpu().numpy()
        all_embeddings.append(embeddings)
    return np.vstack(all_embeddings)

# Define the path for saving/loading embeddings
embeddings_path = './dataset/doc_embeddings.npy'
if os.path.exists(embeddings_path):
    print("Loading embeddings from file...")
    # Load embeddings using memory mapping to avoid loading the entire file into RAM
    doc_embeddings = np.load(embeddings_path, mmap_mode='r')
else:
    print("Generating embeddings in batches...")
    doc_embeddings = embed(documents)
    np.save(embeddings_path, doc_embeddings)  # Save embeddings to disk

# Build the FAISS index using L2 distance
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)
print("FAISS indexing complete!")

Loading embeddings from file...
FAISS indexing complete!


### Batch Inference

In [ ]:
# Retrieve and validate documents
def retrieve_documents(queries, k=3):
    """
    Efficiently retrieves relevant documents for a batch of queries.

    Args:
        queries (list of str): A list of query strings.
        k (int): The number of documents to retrieve per query.

    Returns:
        list of dict: A list where each element corresponds to a query and contains
                      the keys 'ids' and 'contents' for the retrieved documents.
    """
    # Embed all queries at once to avoid redundant computations
    query_embeddings = embed([q.strip().lower() for q in queries])

    # Perform batch search in the index
    distances, indices = index.search(np.array(query_embeddings), k)

    # Retrieve and return documents and their IDs for each query
    results = []
    for query_indices in indices:
        result = {
            "ids": [doc_ids[i] for i in query_indices],
            "contents": [documents[i] for i in query_indices]
        }
        results.append(result)
    return results

# Roughly truncates the input text to a maximum number of characters.
def truncate_text(text, max_chars=4096):
    return text[:max_chars] if len(text) > max_chars else text


async def generate_responses(title_queries, body_queries, contexts):
    """
    Efficiently generates responses for a batch of queries asynchronously.

    Args:
        title_queries (list of str): A list of question titles.
        body_queries (list of str): A list of question bodies.
        contexts (list of str): A list of retrieved knowledge corresponding to each query.

    Returns:
        list: A list of generated responses.
    """

    async def call_openai(prompt):
        """Helper function to call OpenAI API with retries."""
        for attempt in range(3):  # Retry up to 3 times if needed
            try:
                response = await client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=4090,
                    temperature=0
                )
                return response.choices[0].message.content  # Successful response
            except openai.RateLimitError:
                print(f"Rate limit error, retrying ({attempt+1}/3)...")
                await asyncio.sleep(2**attempt)  # Exponential backoff
            except Exception as e:
                print(f"OpenAI API Error: {e}")
                return f"API Error: {e}"  # Store the exact error message
        return "API Error: Max retries exceeded."

    # Prepare prompts using the new structured format
    prompts = [
        f"""
        <prompt>
            <retrieved_knowledge>
                <![CDATA[ {context} ]]>
            </retrieved_knowledge>

            <user_query>
                <title>{title_query}</title>
                <body>{body_query}</body>
            </user_query>

            <instructions>
                <summary>
                    You are a Kubernetes expert and troubleshooting assistant. Your task is to diagnose and resolve Kubernetes-related
                    issues using structured reasoning, verification, and best practices. Use the retrieved knowledge to analyze the problem step-by-step.
                </summary>

                <structured_debugging_approach>
                    <step1>Localization: Identify the exact YAML field, CLI flag, or Kubernetes object causing the issue.</step1>
                    <step2>Reasoning: Explain the precise root cause based on Kubernetes internals, dependencies, and common misconfigurations.</step2>
                    <step3>Remediation: Provide a verified fix (YAML or CLI command) with proper justifications.</step3>
                    <step4>Verification: Ensure that the suggested YAML is syntactically correct, schema-compliant, and valid for Kubernetes.</step4>
                </structured_debugging_approach>

                <problem_solving_strategy>
                    <step1>
                        <title>Reference Retrieved Knowledge First</title>
                        <description>
                            - Analyze the retrieved knowledge above before generating an answer.
                            - If the retrieved knowledge lacks necessary information, apply the fallback strategy.
                        </description>
                    </step1>

                    <step2>
                        <title>Apply Localization → Reasoning → Remediation</title>
                        <description>
                            - <b>Localization:</b> Identify where the issue is occurring (YAML, CLI, or Kubernetes component).
                            - <b>Reasoning:</b> Explain why the issue is occurring using Kubernetes behavior and dependencies.
                            - <b>Remediation:</b> Provide a precise, tested, and verified fix.
                        </description>
                    </step2>

                    <step3>
                        <title>Fallback Strategy for Missing Knowledge</title>
                        <description>
                            - If retrieved knowledge does not contain a direct answer, do not refuse to answer.
                            - Instead, state that the retrieved knowledge is insufficient, then apply Kubernetes best practices to suggest a fix.
                            - Example: "The retrieved knowledge does not provide a direct solution, but based on Kubernetes best practices, here’s a possible fix..."
                        </description>
                    </step3>

                    <step4>
                        <title>YAML Validation Before Output</title>
                        <description>
                            - Before returning a YAML fix, verify its correctness by ensuring:
                                1. **Valid YAML syntax** (correct indentation, structure, and format).
                                2. **Schema compliance** (fields match Kubernetes API structure).
                                3. **API compatibility** (uses the correct apiVersion and supported fields).
                            - If the YAML is incorrect, automatically correct it before returning output.
                            - Example: "The suggested YAML contained an incorrect apiVersion. It has been corrected to apps/v1 before output."
                        </description>
                    </step4>

                    <step5>
                        <title>Chain-of-Thought (CoT) Debugging for Edge Cases</title>
                        <description>
                            - If an explanation is necessary, provide a concise reason first before the YAML/CLI fix.
                            - Example: "This error occurs because apiVersion: apps/v1beta1 is outdated. Use the following YAML instead:"
                        </description>
                    </step5>

                    <step6>
                        <title>Generate a Solution-Driven, Verified Answer</title>
                        <description>
                            - **Issue Summary:** Clearly define the problem based on user input and retrieved knowledge.
                            - **Root Cause Analysis:** Deduce why the issue is occurring step-by-step.
                            - **Step-by-Step Fix:** Provide an exact, actionable YAML or CLI command fix.
                            - **Verification:** Ensure that the solution is correct and aligns with Kubernetes best practices.
                            - **References:** Cite relevant Kubernetes documentation if applicable.
                            - **Common Mistakes & Best Practices:** Highlight potential pitfalls and how to avoid them.
                        </description>
                    </step6>

                    <step7>
                        <title>Ensure a Precise Output Format (YAML or Command Preferred)</title>
                        <description>
                            - If the answer involves configurations, output only the corrected YAML.
                            - If the answer requires CLI commands, output only the correct kubectl commands.
                            - If explanation is needed (edge cases), keep it concise and relevant.
                        </description>
                    </step7>

                    <step8>
                        <title>No Speculation Allowed</title>
                        <description>
                            - If retrieved knowledge is completely lacking, state that and suggest best practices instead.
                            - Only rely on retrieved knowledge, structured reasoning, and Kubernetes best practices.
                        </description>
                    </step8>
                </problem_solving_strategy>

                <output_format>
                    <description>
                        If a YAML or CLI fix is applicable, return only that output. If explanation is needed, keep it concise and relevant.
                    </description>
                </output_format>
            </instructions>
        </prompt>
        """
        for title_query, body_query, context in zip(title_queries, body_queries, contexts)
    ]

    # Run all API calls asynchronously
    tasks = [call_openai(prompt) for prompt in prompts]
    responses = await asyncio.gather(*tasks)

    return responses



async def process_questions(file_path, batch_size=40):
    # Load CSV file into a Pandas DataFrame
    df = pd.read_csv(file_path, encoding="utf-8")

    # Ensure necessary columns exist
    context_col_name = f"{llm_choice}_Top_3_Contexts"
    response_col_name = f"{llm_choice}_Generated_Response"
    context_id_col_name = f"{llm_choice}_Context_IDs"

    if context_col_name not in df.columns:
        df[context_col_name] = ""  # Initialize with empty strings
    if response_col_name not in df.columns:
        df[response_col_name] = ""  # Initialize with empty strings
    if context_id_col_name not in df.columns:
        df[context_id_col_name] = ""  # Initialize with empty strings

    # Get list of unanswered questions
    unanswered_mask = df[f"{llm_choice}_Generated_Response"] == ""
    unanswered_df = df[unanswered_mask]

    # Process questions in batches
    for start in range(0, len(unanswered_df), batch_size):

        batch = unanswered_df.iloc[start : start + batch_size]

        # Since test 2, retrieve body and title seperately.
        title_queries = batch["Question Title"].tolist()
        body_queries = batch["Question Body"].tolist()

        # Retrieve context in batch (each result is a dict with 'ids' and 'contents')
        results = retrieve_documents(body_queries, k=3)

        # For each result, truncate each document's content and join them
        contexts = [
            " ".join([truncate_text(doc) for doc in result["contents"]])
            for result in results
        ]
        # Create context_id strings (concatenated document IDs)
        context_ids = [", ".join([str(doc_id) for doc_id in result['ids']]) for result in results]

        # Generate responses asynchronously
        responses = await generate_responses(title_queries, body_queries, contexts)

        # Update the DataFrame with responses
        df.loc[batch.index, response_col_name] = responses
        df.loc[batch.index, context_col_name] = contexts
        df.loc[batch.index, context_id_col_name] = context_ids
        print(f"Processed {start + len(batch)} / {len(unanswered_df)} questions")


    # Save the updated CSV file with responses
    df.to_csv(file_path, index=False, encoding="utf-8")
    print(f"All questions processed and saved back to {file_path}")

if __name__ == "__main__":
    file_path = "./dataset/test.csv"
    await process_questions(file_path)


embedding 0 / 40
embedding 20 / 40
Processed 40 / 500 questions
embedding 0 / 40
embedding 20 / 40
Processed 80 / 500 questions
embedding 0 / 40
embedding 20 / 40
Processed 120 / 500 questions
embedding 0 / 40
embedding 20 / 40
Processed 160 / 500 questions
embedding 0 / 40
embedding 20 / 40
Processed 200 / 500 questions
embedding 0 / 40
embedding 20 / 40
Processed 240 / 500 questions
embedding 0 / 40
embedding 20 / 40
Processed 280 / 500 questions
embedding 0 / 40
embedding 20 / 40
Processed 320 / 500 questions
embedding 0 / 40
embedding 20 / 40
Processed 360 / 500 questions
embedding 0 / 40
embedding 20 / 40
Processed 400 / 500 questions
embedding 0 / 40
embedding 20 / 40
Processed 440 / 500 questions
embedding 0 / 40
embedding 20 / 40
Processed 480 / 500 questions
embedding 0 / 20
Processed 500 / 500 questions
All questions processed and saved back to ./dataset/test.csv
